<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/sentiment_analysis_lexicons_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment analysis lexicons using Twitter feeds

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/da61b08e7ec945e9b2a5/?dl=1

In [ ]:
!mv index.html?dl=1 lexicon-overlap-score-examples-master.tar.gz
!tar -xvzf lexicon-overlap-score-examples-master.tar.gz

In [ ]:
# -*- coding: utf-8 -*-
# Python script to open each file, read json input and copy to one text file for subsequent processing
import os, re, sys

class clLex2Dict(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University, 2021
    @email bogdan [dot] babych [at] iued [dot] uni-heidelberg [dot] de
    '''
    def __init__(self, SDirName, output_file = 'lexicon-tweets-all.txt', filtr = 0): # initialising by openning the directories
        self.SOutput_file = output_file
        self.Filter = filtr
        self.openDir(SDirName)
        return

    def openDir(self, path): # implementation of recursively openning directories from a given rule directory and reading each file recursively into a string
        i = 0
        DLexiconAll = {}
        FOut = open(self.SOutput_file, 'w')

        for root,d_names,f_names in os.walk(path):
            for f in f_names: # for each file found in the directory
                ## remove this if using on another corpus

                fullpath = os.path.join(root, f)
                i+=1
                if i%1==0: 
                    print(str(i) + '. Processing: ' + f)
                    print(fullpath)

                DLexicon1File = self.procFile(fullpath, f, i) # returns converted string + tags
                
                for key, value in DLexicon1File.items():
                    try:
                        LValues = DLexiconAll[key]
                    except:
                        LValues = []
                    '''
                    '''
                    LValues.append(value)
                    DLexiconAll[key] = LValues
                    del(LValues)
        DLexiconMerged, DLexiconMerged2Conts = self.mergeLex(DLexiconAll)
        self.printDict(DLexiconMerged, DLexiconMerged2Conts, FOut)

        # if SText2Write: FOut.write(SText2Write) # if the string is not empty then write to file
        # FIn.close()

        '''
        try:
            pass
        except:
            print(f'file {f} cannot be read or processed')
        finally:
            pass
        '''
        
        FOut.flush()
        FOut.close()

        return


    def procFile(self, fullpath, SFNameIn, i): # sending each json string for extraction of text and attaching an correct tags to each output string output string
        DLexicon1File = {}
        FIn = open(fullpath, 'r')
        for SLine in FIn:
            SLine = SLine.strip()
            try: 
                LLine = re.split('\t', SLine)
                SWord = LLine[0]
                SSentiment = LLine[1]
                ASentiment = float(SSentiment)
                DLexicon1File[SWord] = ASentiment
            except:
                pass

        return DLexicon1File



    def mergeLex(self, DLexiconAll):
        '''

        '''
        DLexiconMerged = {}
        DLexiconMerged2Conts = {}

        
        for key, value in DLexiconAll.items():
            sum = 0
            for el in value:
                try:
                    sum = sum + el
                except:
                    pass
            average = sum / len(value)
            DLexiconMerged[key] = average
            DLexiconMerged2Conts[key] = len(value)


        return DLexiconMerged, DLexiconMerged2Conts

    def printDict(self, Dict, DictConts, FOut):
        for key, value in sorted(Dict.items() , key=lambda x: x[1], reverse=False):
            counts = DictConts[key]
            if self.Filter:
                if counts > self.Filter:
                    FOut.write(key + '\t' + str(value) + '\t' + str(counts) + '\n')
            else:
                FOut.write(key + '\t' + str(value) + '\t' + str(counts) + '\n')
        

# calling the class
OLexicon = clLex2Dict('/content/lexicon-overlap-score-examples-master/lexicons/', filtr = 1)

# Another positive-negative lexicon
extracting from annotated file, putting into tsv

In [ ]:
!wget https://heibox.uni-heidelberg.de/f/9072fdbed717466db460/?dl=1
!mv index.html?dl=1 subjectivity_clues_hltemnlp05.zip
!unzip subjectivity_clues_hltemnlp05.zip

In [5]:
# global DTwitterLex
def readTweeterLex(FInLex):
    DTwitterLex = {}
    for SLine in FInLex:
        SLine = SLine.strip()
        LLine = re.split('\t', SLine)
        try:
            SWord = LLine[0]
            SWeight = LLine[1]
            DTwitterLex[SWord] = float(SWeight)
        except:
            print('.')
    return DTwitterLex



In [ ]:
FInLex = open('/content/lexicon-tweets-all.txt', 'r')
SFIn = '/content/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff'
SFOut = 'lexicon-positive-negative.txt'
PositiveC = 0
PositiveS = 0

NegativeC = 0
NegativeS = 0

NeutralC = 0
NeutralS = 0

NonFoundC = 0

DTwitterLex = readTweeterLex(FInLex)
print(len(DTwitterLex.items()))
FIn = open(SFIn, 'r')
FOut = open(SFOut, 'w')
for SLine in FIn:
    SLine = SLine.strip()
    LLine = re.split('[ =]', SLine)
    # try:
    SWord = LLine[5]
    SPoS = LLine[7]
    SSentiment = LLine[11]


    # Weight = 'No'
    
    try:
        Weight = DTwitterLex[SWord]
    except:
        Weight = 'NF'

    S2Print = '\t'.join([SWord, SPoS, SSentiment, str(Weight)])
    # except:
    #     pass
    if Weight == 'NF':
        NonFoundC += 1
    elif SSentiment == 'negative':
        NegativeC +=1
        NegativeS += Weight
    elif SSentiment == 'positive':
        PositiveC += 1
        PositiveS += Weight

    elif SSentiment == 'neutral':
        NeutralC +=1
        NeutralS += Weight
       
    FOut.write(S2Print + '\n')
FOut.flush()
FOut.close()

PositiveA = PositiveS/PositiveC
NegativeA = NegativeS/NegativeC
NeutralA = NeutralS/NeutralC

print(PositiveC)
print(PositiveS)
print(PositiveA)

print(NegativeC)
print(NegativeS)
print(NegativeA)

print(NeutralC)
print(NeutralS)
print(NeutralA)

print(NonFoundC)


# Merging both lexicons

In [ ]:
def printDict(Dict, DictT, DictPN, FOut):
    for key, value in sorted(Dict.items() , key=lambda x: x[1], reverse=False):
        valueT = DictT[key]
        valuePN = DictPN[key]
        FOut.write(key + '\t' + str(value) + '\t' + str(valueT) + '\t' + str(valuePN) + '\n')
    FOut.flush()
    FOut.close()
def readTweeterLex(FInLex):
    DTwitterLex = {}
    DTwitterLexCount = {}
    for SLine in FInLex:
        SLine = SLine.strip()
        LLine = re.split('\t', SLine)
        try:
            SWord = LLine[0]
            SWeight = LLine[1]
            SNofTopics = LLine[2]
            DTwitterLex[SWord] = float(SWeight)
            DTwitterLexCount[SWord] = SNofTopics
        except:
            print('.')
    return DTwitterLex, DTwitterLexCount


def readPositiveNegativeLex(SFIn):
    DPositiveNegativeLex = {}
    for SLine in FIn:
        SLine = SLine.strip()
        LLine = re.split('[ =]', SLine)
        # try:
        SWord = LLine[5]
        SPoS = LLine[7]
        SSentiment = LLine[11]
        try:
            DPositiveNegativeLex[SWord] = LValuesPosNeg
        except:
            LValuesPosNeg = []

        LValuesPosNeg.append(SSentiment)
        DPositiveNegativeLex[SWord] = LValuesPosNeg
        del(LValuesPosNeg)


    for key, val in DPositiveNegativeLex.items():
        se = set(val)
        if len(se) > 1:
            print(key)
            print(val)
        DPositiveNegativeLex[key] = list(se)[0]
    return DPositiveNegativeLex


def mergeDicts(DTweet, DPosNeg):
    '''
    we trust manual annotation more, only use Tweet if no item found in manual
    '''
    DAll = {}
    DAllEPN = {}
    DAllETw = {}
    for key, val in DTweet.items():
        if key not in DPosNeg.keys():
            DAll[key] = val
            DAllEPN[key] = False
            DAllETw[key] = val
        else:
            valR = None
            valPosNeg = DPosNeg[key]
            if valPosNeg == 'positive': valR = 1
            elif valPosNeg == 'negative': valR = -1
            elif valPosNeg == 'neutral': valR = 0
            elif valPosNeg == 'both' or valPosNeg == 'priorpolarity': valR = 0 # then we will trust Twitter again?
            DAllEPN[key] = valPosNeg
            DAllETw[key] = val
            DAll[key] = valR
            if valPosNeg == 'both' or valPosNeg == 'priorpolarity': DAll[key] = val # we trust Twitter! (change the value from 0 neutral to twitter!)
            if valR == None: print('Tw: ', key, val) # checks for errors

    for key, val in DPosNeg.items():
        valR = None
        valPosNeg = DPosNeg[key]
        if valPosNeg == 'positive': valR = 1
        elif valPosNeg == 'negative': valR = -1
        elif valPosNeg == 'neutral': valR = 0
        elif valPosNeg == 'both' or valPosNeg == 'priorpolarity': valR = 0 # then we will trust Twitter again?



        if key not in DTweet.keys():
            DAllEPN[key] = valPosNeg
            DAllETw[key] = False

        else:
            valT = DTweet[key]
            DAllEPN[key] = valPosNeg
            DAllETw[key] = valT            

        DAll[key] = valR
        if (valPosNeg == 'both' or valPosNeg == 'priorpolarity') and key in DTweet.keys(): DAll[key] = valT # we trust Twitter! (change the value from 0 neutral to twitter!)
        if valR == None: print('PN: ', key, val) # checks for errors

    return DAll, DAllETw, DAllEPN


FInLex = open('/content/lexicon-tweets-all.txt', 'r')
SFIn = '/content/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff'
FIn = open(SFIn, 'r')
SFOut = 'lexicon-sentiment-tw-n-pn.txt'
FOut = open(SFOut, 'w')

DTwitterLex, DTwitterLexCount = readTweeterLex(FInLex)
print(len(DTwitterLex.items()))
DPositiveNegativeLex = readPositiveNegativeLex(FIn)
print(len(DPositiveNegativeLex.items()))

DAll, DAllEPN, DAllETw = mergeDicts(DTwitterLex, DPositiveNegativeLex)

print(len(DAll))
print(len(DAllETw))
print(len(DAllEPN))

printDict(DAll, DAllETw, DAllEPN, FOut)



In [ ]:
i=0
for key, val in DPositiveNegativeLex.items():
    i+=1
    if i%100 == 0:
        print(key)
        print('\t' + val)


# VADER sentiment repository

In [ ]:
!git clone https://github.com/cjhutto/vaderSentiment.git

In [10]:
SFInput = '/content/vaderSentiment/vaderSentiment/vader_lexicon.txt'
FInput = open(SFInput, 'r')
DVader = {}
for SLine in FInput:
    SLine = SLine.strip()
    LLine = re.split('\t', SLine)
    SWord = LLine[0]
    SScore = LLine[1]
    DVader[SWord] = float(SScore)


In [11]:
# merging all three lexicons: Vader and Tw-n-Pn
SFInput = '/content/lexicon-sentiment-tw-n-pn.txt'
FInput = open(SFInput, 'r')
DTpn = {}
DTpn2 = {}
DTpn3 = {}

for SLine in FInput:
    SLine = SLine.strip()
    LLine = re.split('\t', SLine)
    SWord = LLine[0]
    SScore = LLine[1]
    SF2 = LLine[2]
    SF3  = LLine[3]
    DTpn[SWord] = float(SScore)
    DTpn2[SWord] = SF2
    DTpn3[SWord] = SF3



In [ ]:
def printDict2(FOut, DJointLex, DJointLexV, DJointLexT, DJointLexT2, DJointLexT3):
    for key, value in sorted(DJointLex.items() , key=lambda x: x[1], reverse=False):

        FOut.write(key + '\t' + str(value) + '\t' + str(DJointLexV[key]) + '\t' + str(DJointLexT[key]) + '\t' + str(DJointLexT2[key]) + '\t' + str(DJointLexT3[key]) + '\n')
    FOut.flush()
    FOut.close()


def mergeDicts2(DVaderLex, DTpnLex):
    '''
    we trust VADER more, only use TPN if no item found in VADER
    '''
    DJointLex = {}
    DJointLexV = {}
    DJointLexT = {}
    DJointLexT2 = {}
    DJointLexT3 = {}

    for key, val in DTpnLex.items():
        if key in DVaderLex.keys():
            # integrate information from both dictionaries
            DJointLex[key] = DVaderLex[key] # we trust VADER more!
            DJointLexV[key] =  DVaderLex[key]
            DJointLexT[key] = DTpnLex[key]
            DJointLexT2[key] = DTpn2[key]
            DJointLexT3[key] = DTpn3[key]

        else:
            DJointLex[key] = DTpnLex[key] # we trust VADER more!
            DJointLexV[key] =  False
            DJointLexT[key] = DTpnLex[key]
            DJointLexT2[key] = DTpn2[key]
            DJointLexT3[key] = DTpn3[key]

    for key, val in DVaderLex.items():
        if key in DTpnLex.keys():
            # integrate information from both dictionaries
            DJointLex[key] = DVaderLex[key] # we trust VADER more!
            DJointLexV[key] =  DVaderLex[key]
            DJointLexT[key] = DTpnLex[key]
            DJointLexT2[key] = DTpn2[key]
            DJointLexT3[key] = DTpn3[key]
        else:
            DJointLex[key] = DVaderLex[key] # VADER is the only option!
            DJointLexV[key] =  DVaderLex[key]
            DJointLexT[key] = False
            DJointLexT2[key] = False
            DJointLexT3[key] = False
    
    return DJointLex, DJointLexV, DJointLexT, DJointLexT2, DJointLexT3


SFOut = 'lexicon-sentiment-joint-vader-pn-tw.txt'
FOut = open(SFOut, 'w')

print(len(DVader.items()))
print(len(DTpn.items()))

DJointLex, DJointLexV, DJointLexT, DJointLexT2, DJointLexT3 = mergeDicts2(DVader, DTpn)

print(len(DJointLex))
print(len(DJointLexV))
print(len(DJointLexT))
print(len(DJointLexT2))
print(len(DJointLexT3))

printDict2(FOut, DJointLex, DJointLexV, DJointLexT, DJointLexT2, DJointLexT3)

# Filtering lexicon: 
only manually annotated items from Vader and Positive-Negative lexicons remain

In [16]:
SFIn = '/content/lexicon-sentiment-joint-vader-pn-tw.txt'
SFOut = 'lexicon-sentiment-joint-vader-pn-manual.txt'

FIn = open(SFIn, 'r')
FOut = open(SFOut, 'w')

for SLine in FIn:
    SLine = SLine.strip()
    LLine = re.split('\t', SLine)
    if LLine[2] == 'False' and LLine[4] == 'False':
        continue
    else:
        FOut.write(SLine + '\n')


FOut.flush()
FOut.close()